In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

from src.reader import read_lusa, read_timebank

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
pd.set_option('display.max_rows', 500)

ROOT = Path().resolve().parent.parent

In [2]:
models = [
    'llama2-7b',
    'llama2-7b-chat',
    'llama2-13b',
    'llama2-13b-chat',
    'llama2-70b',
    'llama2-70b-chat',
    'gpt3',
    'chatgpt',
    'gpt4',
]

## Prompt Selection

In [3]:
results_path = ROOT / "results" / "prompt_selection"

In [4]:
df_pt = pd.read_csv(results_path / "portuguese"/ "results.csv")
df_pt["language"] = "Portuguese"

df_en = pd.read_csv(results_path / "english"/ "results.csv")
df_en["language"] = "English"

df = pd.concat([df_pt, df_en])
df.reset_index(inplace=True, drop=True)
df["model"] = pd.Categorical(df.model, ordered=True, categories=models)

In [5]:
languages = df.language.unique().tolist()
entities = df.entity.unique().tolist()
templates = ["ext", "cls",	"ext_def", "ext_exp", "cls_def", "ext_def_exp",  "cls_exp", "cls_def_exp"]
labels = ["_ _ _", "C _ _", "_ D _", "_ _ E", "C D _", "_ D E",  "C _ E", "C D E"]

In [6]:
data = df[["language", "model", "template", "entity", "f1"]].groupby(
    ["language", "entity", "model", "template"]).mean("f1")
data = data.unstack("template")
data = data[[('f1', "ext"),
            ('f1', "cls"),
            ('f1', "ext_def"),
            ('f1', "ext_exp"),
            ('f1', "cls_def"),
            ('f1', "ext_def_exp"),
            ('f1', "cls_exp"),
            ('f1', "cls_def_exp")]]
(data * 100).round(2)

f1                         \
template                                       ext    cls ext_def ext_exp   
language   entity           model                                           
English    event triggers   llama2-7b         0.00   1.34    0.00   10.30   
                            llama2-7b-chat    6.41   0.00   10.90   28.80   
                            llama2-13b        0.00   1.91    2.93    8.85   
                            llama2-13b-chat   2.28   0.31    2.43   20.90   
                            llama2-70b        8.34   3.31   14.96   26.44   
                            llama2-70b-chat   1.17   3.30    2.29   41.22   
                            gpt3              0.30   3.92    3.04   41.12   
                            chatgpt           8.29  33.93   35.96   55.08   
                            gpt4             20.45  57.82   21.36   72.81   
           participants     llama2-7b          NaN    NaN     NaN     NaN   
                            llama2-7b-chat     NaN    NaN     NaN     NaN   
                            llama2-13b         NaN    NaN     NaN     NaN   
                            llama2-13b-chat    NaN    NaN     NaN     NaN   
                            llama2-70b         NaN    NaN     NaN     NaN   
                            llama2-70b-chat    NaN    NaN     NaN     NaN   
                            gpt3               NaN    NaN     NaN     NaN   
                            chatgpt            NaN    NaN     NaN     NaN   
                            gpt4               NaN    NaN     NaN     NaN   
           time expressions llama2-7b         0.00   0.00    0.00    2.12   
                            llama2-7b-chat    7.07   0.00   11.84   10.63   
                            llama2-13b        1.32   0.00    3.30    2.27   
                            llama2-13b-chat  21.09   8.99   22.22   16.62   
                            llama2-70b       12.26   2.91   15.91   21.93   
                            llama2-70b-chat  32.56  16.04   31.16   25.77   
                            gpt3             37.61  27.01   39.17   37.84   
                            chatgpt          35.82  26.38   30.61   42.64   
                            gpt4             58.45  64.19   53.81   64.22   
Portuguese event triggers   llama2-7b         0.00   0.00    9.30   29.64   
                            llama2-7b-chat    3.49   0.00    9.27   24.20   
                            llama2-13b        0.00   2.57   17.56   33.37   
                            llama2-13b-chat   9.12   4.78   13.27   18.21   
                            llama2-70b        3.76   0.78    7.73   31.75   
                            llama2-70b-chat   2.60  10.84    1.57   30.04   
                            gpt3              2.26  22.42    2.80   50.13   
                            chatgpt          33.16  37.91   47.86   52.81   
                            gpt4              6.25  48.14    5.29   62.73   
           participants     llama2-7b         0.00   1.61    6.18   11.41   
                            llama2-7b-chat   10.57   0.00   13.28   10.43   
                            llama2-13b        0.00   0.00   10.53   21.33   
                            llama2-13b-chat   9.28  13.39    8.96   14.09   
                            llama2-70b        8.53   3.45   11.81   16.51   
                            llama2-70b-chat   5.92  16.43    7.39   22.51   
                            gpt3             16.97  19.48   15.72   27.61   
                            chatgpt          12.10  19.12   13.25   36.91   
                            gpt4              8.06  17.16    7.49   52.04   
           time expressions llama2-7b         2.86   0.00    0.00    0.00   
                            llama2-7b-chat    7.71   5.48   10.03    6.55   
                            llama2-13b        6.45   0.00    4.11    0.00   
                            llama2-13b-chat  33.33   6.55   30.67   22.02   
                            llama2-70b      

## Test

In [7]:
results_path = ROOT / "results" / "test"

models = [
    "tei2go",
    "heideltime",
    "tefe",
    "srl",
    "ner",
    "tieval_baseline",
    'llama2-7b',
    'llama2-7b-chat',
    'llama2-13b',
    'llama2-13b-chat',
    'llama2-70b',
    'llama2-70b-chat',
    'gpt3',
    'chatgpt',
    'gpt4',
]

In [8]:
df_pt = pd.read_csv(results_path / "portuguese"/ "results.csv")
df_pt["language"] = "Portuguese"

df_en = pd.read_csv(results_path / "english"/ "results.csv")
df_en["language"] = "English"

df = pd.concat([df_pt, df_en])
df.reset_index(inplace=True, drop=True)
df["model"] = pd.Categorical(df.model, ordered=True, categories=models)

In [9]:
data = df[["language", "entity", "model", "precision", "recall", "f1", "f1_r"]]

data = data.groupby(["language", "entity", "model"]).mean()
(data * 100).round(2)

precision  recall     f1   f1_r
language   entity           model                                           
English    event triggers   tei2go                 NaN     NaN    NaN    NaN
                            heideltime             NaN     NaN    NaN    NaN
                            tefe                   NaN     NaN    NaN    NaN
                            srl                  74.52   73.83  74.17  73.61
                            ner                    NaN     NaN    NaN    NaN
                            tieval_baseline      65.37   65.48  65.43  63.94
                            llama2-7b            21.74   12.50  15.87  11.63
                            llama2-7b-chat       31.20   27.46  29.21  36.78
                            llama2-13b           22.16   11.07  14.77   7.83
                            llama2-13b-chat      20.30   22.92  21.53  22.18
                            llama2-70b           21.93   26.17  23.86  21.46
                            llama2-70b-chat      42.77   41.76  42.26  51.19
                            gpt3                 58.79   38.29  46.37  58.88
                            chatgpt              79.30   36.65  50.14  71.56
                            gpt4                 81.43   57.27  67.25  77.06
           participants     tei2go                 NaN     NaN    NaN    NaN
                            heideltime             NaN     NaN    NaN    NaN
                            tefe                   NaN     NaN    NaN    NaN
                            srl                   0.00    0.00   0.00   0.00
                            ner                    NaN     NaN    NaN    NaN
                            tieval_baseline        NaN     NaN    NaN    NaN
                            llama2-7b              NaN     NaN    NaN    NaN
                            llama2-7b-chat         NaN     NaN    NaN    NaN
                            llama2-13b             NaN     NaN    NaN    NaN
                            llama2-13b-chat        NaN     NaN    NaN    NaN
                            llama2-70b             NaN     NaN    NaN    NaN
                            llama2-70b-chat        NaN     NaN    NaN    NaN
                            gpt3                   NaN     NaN    NaN    NaN
                            chatgpt                NaN     NaN    NaN    NaN
                            gpt4                   NaN     NaN    NaN    NaN
           time expressions tei2go               87.07   71.44  78.49  88.24
                            heideltime           85.90   70.72  77.57  87.90
                            tefe                   NaN     NaN    NaN    NaN
                            srl                  28.74   24.03  26.17  59.29
                            ner                    NaN     NaN    NaN    NaN
                            tieval_baseline        NaN     NaN    NaN    NaN
                            llama2-7b             2.42    3.26   2.78   2.86
                            llama2-7b-chat       11.12   18.22  13.81  23.51
                            llama2-13b            3.99    1.36   2.03   5.65
                            llama2-13b-chat      30.89   30.19  30.54  46.49
                            llama2-70b           20.89   21.21  21.05  39.54
                            llama2-70b-chat      33.66   37.17  35.33  54.80
                            gpt3                 40.13   40.71  40.41  58.87
                            chatgpt              55.54   36.36  43.95  61.90
                            gpt4                 69.71   58.84  63.82  81.93
Portuguese event triggers   tei2go                 NaN     NaN    NaN    NaN
                            heideltime             NaN     NaN    NaN    NaN
                            tefe                 94.35    9.64  17.49  95.08
                            srl                  73.43   49.47  59.11  81.15
                            ner                    NaN     NaN    NaN    NaN
                            tieval_baseli